# Introduction

Need to convert <1TB of data per day without the need for specialized hardware and with very limited local hardware. 

Examples (`dtype==uint8`): 1k^3 (`1GB`); 2k^3(`8GB`); 4K^3 (`64GB`); 8k^3 (`512GB`).

The IO activity is related to:
- *input device*  running read-only operations of middle-size files (i.e. some megabytes). Could be network, object storage or USB drive.
- *output device* executing **random** read/write of blocks. Each block has typically size 8KB-64KB (equivalent to 2^13 -  2^16 samples). We do not want blocks too large since we need them to be network optimized (ip max size is 64KB) and could cause excessive unused samples.

Consider this example: `4096^3` uint8 dataset (`64GB`) with blocksize `64KB`.  

We can convert the dataset following three different approaches:

- **write-single-slices**: we convert single z-axis aligned slices images. For each slice, we need about 10k IO operations.  The total number of written bytes is 4096*10k*64KB=2.5TB (ratio 1:39).  Assuming a 300 IOPS (in random writes) disk it would take 37 hours just for the IO writing part, without taking into account that (i) the same amount of blocks need to be read to merge pre-existing samples and (ii) there are several file-seeks to read and update the file headers (iii) we probably need to compress the blocks at the end.
- **write-slabs** we collect several z-axis aligned images in a slab. For example, for each slab of 16 slices (16MB*16=256MB), we need about 20K IO operations.  The total number of written bytes is  (4096/16)*20K*64KB=312GB (ratio 1:5). Assuming a 300 IOPS disk it would take **4 hours** just for the IO writing part.

- **write-bricks** we convert well-aligned and _uniform_ data bricks; either the logic domain and the bricks are power-of-two and multiple of each other. For example, to write a brick of 256^3 samples (16MB) we need 272 IO operations.  The total number of written bytes is (4096/256)^3*272*64KB=66GB written on disk (ratio 1:1). Assuming a 300 IOPS disk it would take **1 hour** just for the IO writing part.

<img src="https://www.researchgate.net/profile/Ilia-Petrov-2/publication/262402709/figure/fig1/AS:296743922094097@1447760672824/Random-throughput-IOPS-of-a-X25-E-SSD-vs-HDD-7200-RPM.png">



In [14]:
import sys
import numpy as np

sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
from OpenVisus import *

KB,MB,GB=1024,1024*1024,1024*1024*1024

def Assert(cond):
	if not cond: raise Exception("error")

Create a fake dataset

In [12]:
dtype="uint8[1]"
dims=[4096,4096,4096]
field=Field("data", dtype,"row_major")
db=CreateIdx(url='D:/~tmp/visus-convert-speed-test/visus.idx', rmtree=True, dims=dims,fields=[field])
print(db.getDatasetBody().toString())

<dataset url="D:/~tmp/visus-convert-speed-test/visus.idx" typename="IdxDataset">
	<idxfile>
		<version value="6" />
		<bitmask value="V012012012012012012012012012012012012" />
		<box value="0 4096 0 4096 0 4096" />
		<bitsperblock value="16" />
		<blocksperfile value="512" />
		<block_interleaving value="0" />
		<filename_template value="./visus/%02x/%04x.bin" />
		<missing_blocks value="False" />
		<time_template value="" />
		<field name="data" description="" index="" default_compression="" default_layout="default_value(0" default_value="0" filter="" dtype="uint8" />
		<timestep when="0" />
	</idxfile>
</dataset>


Create an access just to simulate read/writing  (I am only interested in the IO number of blocks)

In [15]:
# fake reading/writing just for statistics
access = IdxDiskAccess.create(db)
access.disableAsync() # sync access 
access.disableWriteLock() # no write lock
access.setSkipReading(True) # no reading
access.setSkipWriting(True) # no writing

samples_per_block=access.getSamplesPerBlock()
print("samples_per_block",samples_per_block)

blocksize=db.getField().dtype.getByteSize(samples_per_block)
print("blocksize",blocksize)
Assert(blocksize==field.dtype.getByteSize(samples_per_block))

samples_per_block 65536
blocksize 65536


# write slice by slice (worst case)

In [49]:
# hypothesis of disk benchmarks for random read/write
write_iops=300
read_iops=write_iops*10

def BeginStatistics():
    access.statistics.reset()
    visus_memory=RamResource.getSingleton().getVisusUsedMemory()
    RamResource.getSingleton().setPeakMemory(visus_memory)

def PrintStatistics(nread,nwrite):
    totread=np.sum(nread)
    totwrite=np.sum(nwrite)
    eta=totread/read_iops + totread/write_iops
    print("\nStatistics:")
    print("\tnread-median",np.median(nread))
    print("\tnwrite-median",np.median(nwrite))
    print("\ttot-read",totread)
    print("\ttot-write",totwrite)
    print("\teta-seconds",eta)
    print("\teta-minutes",eta/60)
    print("\teta-hours",eta/3600)
    print("\tratio", float(totwrite)*blocksize/float(field.dtype.getByteSize(dims[0]*dims[1]*dims[2])))

In [26]:
# create a zero buffer to mimic the slice writing
data=np.zeros((dims[1], dims[0]),dtype=convert_dtype(field.dtype.toString()))

nread,nwrite=[],[]
access.beginWrite()
for z in range(dims[2]):
    
    BeginStatistics()
    
    # write the single slice
    db.write(z=z,access=access,data=data) 
    
    nread.append(access.statistics.rok)
    nwrite.append(access.statistics.wok)
    print("{}/{}".format(z,dims[2]),"nread",nread[-1],"nwrite",nwrite[-1])

access.endWrite()
PrintStatistics(nread, nwrite)

0/4096 num_blocks_read 10922 wok 10922
1/4096 num_blocks_read 10922 wok 10922
2/4096 num_blocks_read 10922 wok 10922
3/4096 num_blocks_read 10922 wok 10922
4/4096 num_blocks_read 10922 wok 10922
5/4096 num_blocks_read 10922 wok 10922
6/4096 num_blocks_read 10922 wok 10922
7/4096 num_blocks_read 10922 wok 10922
8/4096 num_blocks_read 10922 wok 10922
9/4096 num_blocks_read 10922 wok 10922
10/4096 num_blocks_read 10922 wok 10922
11/4096 num_blocks_read 10922 wok 10922
12/4096 num_blocks_read 10922 wok 10922
13/4096 num_blocks_read 10922 wok 10922
14/4096 num_blocks_read 10922 wok 10922
15/4096 num_blocks_read 10922 wok 10922
16/4096 num_blocks_read 10922 wok 10922
17/4096 num_blocks_read 10922 wok 10922
18/4096 num_blocks_read 10922 wok 10922
19/4096 num_blocks_read 10922 wok 10922
20/4096 num_blocks_read 10922 wok 10922
21/4096 num_blocks_read 10922 wok 10922
22/4096 num_blocks_read 10922 wok 10922
23/4096 num_blocks_read 10922 wok 10922
24/4096 num_blocks_read 10922 wok 10922
25/4096 nu

204/4096 num_blocks_read 10922 wok 10922
205/4096 num_blocks_read 10922 wok 10922
206/4096 num_blocks_read 10922 wok 10922
207/4096 num_blocks_read 10922 wok 10922
208/4096 num_blocks_read 10922 wok 10922
209/4096 num_blocks_read 10922 wok 10922
210/4096 num_blocks_read 10922 wok 10922
211/4096 num_blocks_read 10922 wok 10922
212/4096 num_blocks_read 10922 wok 10922
213/4096 num_blocks_read 10922 wok 10922
214/4096 num_blocks_read 10922 wok 10922
215/4096 num_blocks_read 10922 wok 10922
216/4096 num_blocks_read 10922 wok 10922
217/4096 num_blocks_read 10922 wok 10922
218/4096 num_blocks_read 10922 wok 10922
219/4096 num_blocks_read 10922 wok 10922
220/4096 num_blocks_read 10922 wok 10922
221/4096 num_blocks_read 10922 wok 10922
222/4096 num_blocks_read 10922 wok 10922
223/4096 num_blocks_read 10922 wok 10922
224/4096 num_blocks_read 10922 wok 10922
225/4096 num_blocks_read 10922 wok 10922
226/4096 num_blocks_read 10922 wok 10922
227/4096 num_blocks_read 10922 wok 10922
228/4096 num_blo

404/4096 num_blocks_read 10922 wok 10922
405/4096 num_blocks_read 10922 wok 10922
406/4096 num_blocks_read 10922 wok 10922
407/4096 num_blocks_read 10922 wok 10922
408/4096 num_blocks_read 10922 wok 10922
409/4096 num_blocks_read 10922 wok 10922
410/4096 num_blocks_read 10922 wok 10922
411/4096 num_blocks_read 10922 wok 10922
412/4096 num_blocks_read 10922 wok 10922
413/4096 num_blocks_read 10922 wok 10922
414/4096 num_blocks_read 10922 wok 10922
415/4096 num_blocks_read 10922 wok 10922
416/4096 num_blocks_read 10922 wok 10922
417/4096 num_blocks_read 10922 wok 10922
418/4096 num_blocks_read 10922 wok 10922
419/4096 num_blocks_read 10922 wok 10922
420/4096 num_blocks_read 10922 wok 10922
421/4096 num_blocks_read 10922 wok 10922
422/4096 num_blocks_read 10922 wok 10922
423/4096 num_blocks_read 10922 wok 10922
424/4096 num_blocks_read 10922 wok 10922
425/4096 num_blocks_read 10922 wok 10922
426/4096 num_blocks_read 10922 wok 10922
427/4096 num_blocks_read 10922 wok 10922
428/4096 num_blo

604/4096 num_blocks_read 10922 wok 10922
605/4096 num_blocks_read 10922 wok 10922
606/4096 num_blocks_read 10922 wok 10922
607/4096 num_blocks_read 10922 wok 10922
608/4096 num_blocks_read 10922 wok 10922
609/4096 num_blocks_read 10922 wok 10922
610/4096 num_blocks_read 10922 wok 10922
611/4096 num_blocks_read 10922 wok 10922
612/4096 num_blocks_read 10922 wok 10922
613/4096 num_blocks_read 10922 wok 10922
614/4096 num_blocks_read 10922 wok 10922
615/4096 num_blocks_read 10922 wok 10922
616/4096 num_blocks_read 10922 wok 10922
617/4096 num_blocks_read 10922 wok 10922
618/4096 num_blocks_read 10922 wok 10922
619/4096 num_blocks_read 10922 wok 10922
620/4096 num_blocks_read 10922 wok 10922
621/4096 num_blocks_read 10922 wok 10922
622/4096 num_blocks_read 10922 wok 10922
623/4096 num_blocks_read 10922 wok 10922
624/4096 num_blocks_read 10922 wok 10922
625/4096 num_blocks_read 10922 wok 10922
626/4096 num_blocks_read 10922 wok 10922
627/4096 num_blocks_read 10922 wok 10922
628/4096 num_blo

804/4096 num_blocks_read 10922 wok 10922
805/4096 num_blocks_read 10922 wok 10922
806/4096 num_blocks_read 10922 wok 10922
807/4096 num_blocks_read 10922 wok 10922
808/4096 num_blocks_read 10922 wok 10922
809/4096 num_blocks_read 10922 wok 10922
810/4096 num_blocks_read 10922 wok 10922
811/4096 num_blocks_read 10922 wok 10922
812/4096 num_blocks_read 10922 wok 10922
813/4096 num_blocks_read 10922 wok 10922
814/4096 num_blocks_read 10922 wok 10922
815/4096 num_blocks_read 10922 wok 10922
816/4096 num_blocks_read 10922 wok 10922
817/4096 num_blocks_read 10922 wok 10922
818/4096 num_blocks_read 10922 wok 10922
819/4096 num_blocks_read 10922 wok 10922
820/4096 num_blocks_read 10922 wok 10922
821/4096 num_blocks_read 10922 wok 10922
822/4096 num_blocks_read 10922 wok 10922
823/4096 num_blocks_read 10922 wok 10922
824/4096 num_blocks_read 10922 wok 10922
825/4096 num_blocks_read 10922 wok 10922
826/4096 num_blocks_read 10922 wok 10922
827/4096 num_blocks_read 10922 wok 10922
828/4096 num_blo

...

3160/4096 num_blocks_read 10922 wok 10922
3161/4096 num_blocks_read 10922 wok 10922
3162/4096 num_blocks_read 10922 wok 10922
3163/4096 num_blocks_read 10922 wok 10922
3164/4096 num_blocks_read 10922 wok 10922
3165/4096 num_blocks_read 10922 wok 10922
3166/4096 num_blocks_read 10922 wok 10922
3167/4096 num_blocks_read 10922 wok 10922
3168/4096 num_blocks_read 10922 wok 10922
3169/4096 num_blocks_read 10922 wok 10922
3170/4096 num_blocks_read 10922 wok 10922
3171/4096 num_blocks_read 10922 wok 10922
3172/4096 num_blocks_read 10922 wok 10922
3173/4096 num_blocks_read 10922 wok 10922
3174/4096 num_blocks_read 10922 wok 10922
3175/4096 num_blocks_read 10922 wok 10922
3176/4096 num_blocks_read 10922 wok 10922
3177/4096 num_blocks_read 10922 wok 10922
3178/4096 num_blocks_read 10922 wok 10922
3179/4096 num_blocks_read 10922 wok 10922
3180/4096 num_blocks_read 10922 wok 10922
3181/4096 num_blocks_read 10922 wok 10922
3182/4096 num_blocks_read 10922 wok 10922
3183/4096 num_blocks_read 10922 wo

3356/4096 num_blocks_read 10922 wok 10922
3357/4096 num_blocks_read 10922 wok 10922
3358/4096 num_blocks_read 10922 wok 10922
3359/4096 num_blocks_read 10922 wok 10922
3360/4096 num_blocks_read 10922 wok 10922
3361/4096 num_blocks_read 10922 wok 10922
3362/4096 num_blocks_read 10922 wok 10922
3363/4096 num_blocks_read 10922 wok 10922
3364/4096 num_blocks_read 10922 wok 10922
3365/4096 num_blocks_read 10922 wok 10922
3366/4096 num_blocks_read 10922 wok 10922
3367/4096 num_blocks_read 10922 wok 10922
3368/4096 num_blocks_read 10922 wok 10922
3369/4096 num_blocks_read 10922 wok 10922
3370/4096 num_blocks_read 10922 wok 10922
3371/4096 num_blocks_read 10922 wok 10922
3372/4096 num_blocks_read 10922 wok 10922
3373/4096 num_blocks_read 10922 wok 10922
3374/4096 num_blocks_read 10922 wok 10922
3375/4096 num_blocks_read 10922 wok 10922
3376/4096 num_blocks_read 10922 wok 10922
3377/4096 num_blocks_read 10922 wok 10922
3378/4096 num_blocks_read 10922 wok 10922
3379/4096 num_blocks_read 10922 wo

3552/4096 num_blocks_read 10922 wok 10922
3553/4096 num_blocks_read 10922 wok 10922
3554/4096 num_blocks_read 10922 wok 10922
3555/4096 num_blocks_read 10922 wok 10922
3556/4096 num_blocks_read 10922 wok 10922
3557/4096 num_blocks_read 10922 wok 10922
3558/4096 num_blocks_read 10922 wok 10922
3559/4096 num_blocks_read 10922 wok 10922
3560/4096 num_blocks_read 10922 wok 10922
3561/4096 num_blocks_read 10922 wok 10922
3562/4096 num_blocks_read 10922 wok 10922
3563/4096 num_blocks_read 10922 wok 10922
3564/4096 num_blocks_read 10922 wok 10922
3565/4096 num_blocks_read 10922 wok 10922
3566/4096 num_blocks_read 10922 wok 10922
3567/4096 num_blocks_read 10922 wok 10922
3568/4096 num_blocks_read 10922 wok 10922
3569/4096 num_blocks_read 10922 wok 10922
3570/4096 num_blocks_read 10922 wok 10922
3571/4096 num_blocks_read 10922 wok 10922
3572/4096 num_blocks_read 10922 wok 10922
3573/4096 num_blocks_read 10922 wok 10922
3574/4096 num_blocks_read 10922 wok 10922
3575/4096 num_blocks_read 10922 wo

3748/4096 num_blocks_read 10922 wok 10922
3749/4096 num_blocks_read 10922 wok 10922
3750/4096 num_blocks_read 10922 wok 10922
3751/4096 num_blocks_read 10922 wok 10922
3752/4096 num_blocks_read 10922 wok 10922
3753/4096 num_blocks_read 10922 wok 10922
3754/4096 num_blocks_read 10922 wok 10922
3755/4096 num_blocks_read 10922 wok 10922
3756/4096 num_blocks_read 10922 wok 10922
3757/4096 num_blocks_read 10922 wok 10922
3758/4096 num_blocks_read 10922 wok 10922
3759/4096 num_blocks_read 10922 wok 10922
3760/4096 num_blocks_read 10922 wok 10922
3761/4096 num_blocks_read 10922 wok 10922
3762/4096 num_blocks_read 10922 wok 10922
3763/4096 num_blocks_read 10922 wok 10922
3764/4096 num_blocks_read 10922 wok 10922
3765/4096 num_blocks_read 10922 wok 10922
3766/4096 num_blocks_read 10922 wok 10922
3767/4096 num_blocks_read 10922 wok 10922
3768/4096 num_blocks_read 10922 wok 10922
3769/4096 num_blocks_read 10922 wok 10922
3770/4096 num_blocks_read 10922 wok 10922
3771/4096 num_blocks_read 10922 wo

3944/4096 num_blocks_read 10922 wok 10922
3945/4096 num_blocks_read 10922 wok 10922
3946/4096 num_blocks_read 10922 wok 10922
3947/4096 num_blocks_read 10922 wok 10922
3948/4096 num_blocks_read 10922 wok 10922
3949/4096 num_blocks_read 10922 wok 10922
3950/4096 num_blocks_read 10922 wok 10922
3951/4096 num_blocks_read 10922 wok 10922
3952/4096 num_blocks_read 10922 wok 10922
3953/4096 num_blocks_read 10922 wok 10922
3954/4096 num_blocks_read 10922 wok 10922
3955/4096 num_blocks_read 10922 wok 10922
3956/4096 num_blocks_read 10922 wok 10922
3957/4096 num_blocks_read 10922 wok 10922
3958/4096 num_blocks_read 10922 wok 10922
3959/4096 num_blocks_read 10922 wok 10922
3960/4096 num_blocks_read 10922 wok 10922
3961/4096 num_blocks_read 10922 wok 10922
3962/4096 num_blocks_read 10922 wok 10922
3963/4096 num_blocks_read 10922 wok 10922
3964/4096 num_blocks_read 10922 wok 10922
3965/4096 num_blocks_read 10922 wok 10922
3966/4096 num_blocks_read 10922 wok 10922
3967/4096 num_blocks_read 10922 wo

# write slabs (better)

In [36]:
# how many slices
slabsize=32
slabsamples=dims[0]*dims[1]*slabsize
print("slabsize",slabsize, "slabsamples",slabsamples,"MBytes",field.dtype.getByteSize(slabsamples)/MB)

slices=[np.zeros((dims[1], dims[0]),dtype=convert_dtype(field.dtype.toString())) for I in range(slabsize)]

nread,nwrite=[],[]
access.beginWrite()
for z in range(0,dims[2],slabsize):

    BeginStatistics()
    db.writeSlabs(slices, z=z, access=access) 

    nread.append(access.statistics.rok)
    nwrite.append(access.statistics.wok)
    print("{}/{}".format(z,dims[2]),"nread",nread[-1],"nwrite",nwrite[-1])

access.endWrite()
PrintStatistics(nread, nwrite)

slabsize 32 slabsamples 512
MBytes 512
<VisusKernelPy.DType; proxy of <Swig Object of type 'Visus::DType *' at 0x0000018382009BD0> >
0/4096 nread 21834 nwrite 21834
32/4096 nread 21832 nwrite 21832
64/4096 nread 21834 nwrite 21834
96/4096 nread 21832 nwrite 21832
128/4096 nread 21834 nwrite 21834
160/4096 nread 21832 nwrite 21832
192/4096 nread 21834 nwrite 21834
224/4096 nread 21832 nwrite 21832
256/4096 nread 21834 nwrite 21834
288/4096 nread 21832 nwrite 21832
320/4096 nread 21834 nwrite 21834
352/4096 nread 21832 nwrite 21832
384/4096 nread 21834 nwrite 21834
416/4096 nread 21832 nwrite 21832
448/4096 nread 21834 nwrite 21834
480/4096 nread 21832 nwrite 21832
512/4096 nread 21834 nwrite 21834
544/4096 nread 21832 nwrite 21832
576/4096 nread 21834 nwrite 21834
608/4096 nread 21832 nwrite 21832
640/4096 nread 21834 nwrite 21834
672/4096 nread 21832 nwrite 21832
704/4096 nread 21834 nwrite 21834
736/4096 nread 21832 nwrite 21832
768/4096 nread 21834 nwrite 21834
800/4096 nread 21832 n

<ipython-input-27-64fd5ac75c0d>:17: RuntimeWarning: overflow encountered in long_scalars
  print("\tratio", totwrite*blocksize/field.dtype.getByteSize(dims))


TypeError: Wrong number or type of arguments for overloaded function 'DType_getByteSize'.
  Possible C/C++ prototypes are:
    Visus::DType::getByteSize() const
    Visus::DType::getByteSize(Visus::Int64) const
    Visus::DType::getByteSize(Visus::PointNi) const


# Write bricks (best)

In [48]:
brick=(256,256,256) # 16MG of data
print("Bricks",brick,"MBytes",field.dtype.getByteSize(brick[0]*brick[1]*brick[2])/MB)
data=np.zeros((brick[2], brick[1],brick[1]),dtype=convert_dtype(field.dtype.toString()))

nread,nwrite=[],[]
access.beginWrite()
for z in range(0,dims[2],brick[2]):
    for y in range(0,dims[1],brick[1]):
        for x in range(0,dims[0],brick[0]):

            BeginStatistics()

            # write the brick
            db.write(data=data, x=x,y=y,z=z, access=access) 
            
            nread.append(access.statistics.rok)
            nwrite.append(access.statistics.wok)
            
            print("{}/{}".format(z,dims[2]),"nread",nread[-1],"nwrite",nwrite[-1])

access.endWrite()
PrintStatistics(nread, nwrite)

Bricks (256, 256, 256) MBytes 16777216
0/4096 nread 268 nwrite 268
256/4096 nread 268 nwrite 268
512/4096 nread 268 nwrite 268
768/4096 nread 268 nwrite 268
1024/4096 nread 268 nwrite 268
1280/4096 nread 268 nwrite 268
1536/4096 nread 268 nwrite 268
1792/4096 nread 268 nwrite 268
2048/4096 nread 268 nwrite 268
2304/4096 nread 268 nwrite 268
2560/4096 nread 268 nwrite 268
2816/4096 nread 268 nwrite 268
3072/4096 nread 268 nwrite 268
3328/4096 nread 268 nwrite 268
3584/4096 nread 268 nwrite 268
3840/4096 nread 268 nwrite 268

Statistics:
	nread-median 268.0
	nwrite-median 268.0
	tot-read 1097728
	tot-write 1097728
	eta-seconds 4025.0026666666668
	eta-minutes 67.08337777777778
	eta-hours 1.1180562962962963


TypeError: Wrong number or type of arguments for overloaded function 'DType_getByteSize'.
  Possible C/C++ prototypes are:
    Visus::DType::getByteSize() const
    Visus::DType::getByteSize(Visus::Int64) const
    Visus::DType::getByteSize(Visus::PointNi) const


# Mosaic MIDX

(TODO) interesting case worth investing for parallel conversion with multiple nodes, with multiple disks to circunvent the IOPS problem.
Create several IDX in mosaic-mode and at the end collect-rewrite samples only on the first coarse levels.

Example: with 4 workers we can get 4x speed up apart for final reduce step (in a map-shuffle-reduce)
